In [ ]:
import os
import string
import re

import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load("en_core_web_md")

2021-07-09 04:45:50.620337: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-09 04:45:50.620374: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
data_path = "../../data_collection/data/processed"
pgm_india = pd.read_csv(os.path.join(data_path, "india_pgms.csv"))
pgm_usa = pd.read_csv(os.path.join(data_path, "usa_pgms.csv"))

curr_india = pd.read_csv(os.path.join(data_path, "india_curr.csv"))
curr_usa = pd.read_csv(os.path.join(data_path, "usa_curr.csv"))
curr_usa = curr_usa.drop(['Unnamed: 4', 'Unnamed: 5'], axis=1)

In [ ]:
pgm_india.head(2)

,uni_name,pgm_name,dur_yrs,type,national_rank_qs,dept_involved,location,url,descr,header_tag,header_names,links,date_pub,date_last_mod,html_tags,html_tag_freq,html_emphasized_elements
0,Indian Institute of Science Bangalore,M.Tech. Computational and Data Science,2,Public,2,Department of Computational and Data Sciences,Karnataka,http://cds.iisc.ac.in/admissions/course-degrees/,Computational Science and Data Science are int...,"['h1', 'h2', 'h3']","[['\n\t\t\tCourse Degree Admissions, 2021\t\t'...",['https://iisc.ac.in/admissions/m-tech-m-des/'...,Not inferred,Not inferred,"['header', 'h1', 'div', 'ul', 'li', 'span', 's...","{'header': 1, 'h1': 1, 'div': 1, 'ul': 6, 'li'...","['the m.tech. (cds) admission process', 'downl..."
1,Indian Institute of Science Bangalore,M.Tech. Artificial Intelligence,2,Public,2,Departments of Computer Science and Automation...,Karnataka,https://eecs.iisc.ac.in/mtechai/,Artificial Intelligence has captured the ima...,"['h1', 'h2']","[['MTech (AI) @ IISc'], ['Admissions 2021 – Ap...","['https://ai.iisc.ac.in', 'https://admissions-...",Not inferred,Not inferred,"['h1', 'p', 'img', 'p', 'strong', 'a', 'p', 'h...","{'h1': 1, 'p': 18, 'img': 2, 'strong': 15, 'a'...",['written test will be held online with remote...


In [ ]:
curr_india.head(2)

,url,compulsory course,course outcome or overview,topics covered
0,http://cds.iisc.ac.in/admissions/course-degrees/,Introduction to Scalable Systems,This course covers computer systems topics tha...,"Architecture: computer organization, single-co..."
1,http://cds.iisc.ac.in/admissions/course-degrees/,Numerical Linear Algebra,NaN,"Matrix and vector norms, floating points arith..."


In [ ]:
program_descriptions_india = pgm_india.descr.fillna('')
program_descriptions_usa = pgm_usa.descr.fillna('')
aggregated_program_descriptions = ' '.join(program_descriptions_india) + ' '.join(program_descriptions_usa)

In [ ]:
len(aggregated_program_descriptions)

145972

In [ ]:
compulsory_courses_india = curr_india['compulsory course'].fillna('')
compulsory_courses_usa = curr_usa['compulsory course'].fillna('')
aggregated_compulsory_courses = list(compulsory_courses_india) + list(compulsory_courses_usa)

In [ ]:
print(aggregated_compulsory_courses[:10])
print(len(aggregated_compulsory_courses))

['Introduction to Scalable Systems', 'Numerical Linear Algebra', 'Numerical Methods', 'Data Analysis and Visualization', 'Research Methods - Softs Skills Course', 'Data Structures and Algorithms', 'Stochastic Models and Applications', 'Random Processes', 'Computational Linear Algebra', 'Computational Methods of Optimization']
1002


- There are 1002 unique compulsory courses in the dataset (both years combined)
- Run these through the labeller
    - Ensure randomization to not be biased while labelling

In [ ]:
class TextPreprocessor:

    def __init__(self, text):
        self.text = text

    def make_lowercase(self):
        self.text = self.text.lower()

    def remove_numbers(self):
        self.text = self.text.translate({ord(k): None for k in string.digits})

    def remove_unnecessary_whitespace(self):
        self.text = self.text.replace('\t', '').replace('\n', '')
        self.text = re.sub(' +', ' ', self.text)
        self.text = self.text.strip()

    def remove_urls(self):
        self.text = re.sub(r'http\S+', '', self.text)

    def remove_punctuations(self):
        self.text = self.text.translate({ord(k): None for k in string.punctuation})

    def remove_unicode_chars(self):
        self.text = self.text.encode('ascii', 'ignore').decode('utf-8')

    def preprocess_text(self):

        self.make_lowercase()
        self.remove_numbers()
        self.remove_punctuations()
        self.remove_urls()
        self.remove_unicode_chars()
        self.remove_unnecessary_whitespace()

        return self.text

- Some times, words that don't make sense such as 'di' and 'dames' are introduced post preprocessing

In [ ]:
p = TextPreprocessor(aggregated_program_descriptions)
preprocessed_aggregated_program_descriptions = p.preprocess_text()

In [ ]:
len(preprocessed_aggregated_program_descriptions.split()) # all tokens

20669

In [ ]:
noun_chunks = []
doc = nlp(preprocessed_aggregated_program_descriptions)
for chunk in doc.noun_chunks:
    noun_chunks.append(chunk.text)
noun_chunks = list(set(noun_chunks))

In [ ]:
print(noun_chunks[:10])
print(len(noun_chunks))

['msba program trains', 'the potential', 'individual behaviors', 'an additional trained personnel', 'the option', 'office', 'a phd degree', 'operational decisions', 'human capital analytics and technology functions building', 'fundamental ways']
3374


In [ ]:
nouns_verbs_propernouns = []
doc = nlp(preprocessed_aggregated_program_descriptions)
for token in doc:
    token_text = token.text
    token_pos = token.pos_
    if token_pos in ['PROPN', 'NOUN', 'VERB']:
        nouns_verbs_propernouns.append(token_text)

nouns_verbs_propernouns = list(set(nouns_verbs_propernouns))

In [ ]:
print(nouns_verbs_propernouns[:10])
print(len(nouns_verbs_propernouns))

['shaping', 'council', 'simulation', 'diagnose', 'levels', 'expectancy', 'potential', 'office', 'problemsanalytics', 'fulfills']
2325


### Working with compulsory courses

In [ ]:
print(len(aggregated_compulsory_courses))

1002


In [ ]:
class CourseNamesProcessor:

    def __init__(self, course_list):
        self.courses = course_list

    def remove_duplicates(self):
        self.courses =  list(set(self.courses))

    def remove_punctuations(self, course):
        return course.translate({ord(k): None for k in string.punctuation})

    def remove_numbers(self, course):
        return course.translate({ord(k): None for k in string.digits})

    def preprocess_course_names(self):
        self.courses = [course_name.lower().strip() for course_name in self.courses if len(course_name)!=0]
        self.courses = [self.remove_numbers(course_name) for course_name in self.courses]
        self.courses = [self.remove_punctuations(course_name) for course_name in self.courses]

    def remove_stopwords_in_course_names(self):
        stopword_free_courses = []
        temp = " <COURSE SEPARATOR> ".join(self.courses)
        for word in temp.split():
            if not(word in STOP_WORDS):
                stopword_free_courses.append(word)

        self.courses = " ".join(stopword_free_courses).split(" <COURSE SEPARATOR> ")

    def process_course_names(self):

        self.remove_duplicates()
        self.preprocess_course_names()
        self.remove_stopwords_in_course_names()
        self.remove_duplicates()

    def get_courses(self):

        self.process_course_names()

        return (self.courses)

In [ ]:
c = CourseNamesProcessor(aggregated_compulsory_courses)
a = c.get_courses()

In [ ]:
len(a)

787

In [ ]:
join_a = " ".join(a)
join_a_words_unique = set(join_a.split())
print(join_a_words[:10])
print(len(join_a_words_unique))

['relational', 'database', 'management', 'lab', 'management', 'action', 'social', 'economic', 'ethical', 'issues']
498


In [ ]:
a[:20]

['relational database management lab',
 'management action social economic ethical issues',
 'public management',
 'finance accounting',
 'coding civic data applications',
 'financial accounting reporting',
 'business analytics skills workshops',
 'statistical learning',
 'building products deep learning',
 'problem solving computer programming c',
 'advanced statistical inference',
 'advanced data structures',
 'digital health applications',
 'research tools techniques',
 'introduction machine learning',
 'stochastic models applications',
 'pythonprogramming banking finance',
 'programming spatial sciences',
 'introduction data mining',
 'nosql databases big data']

In [ ]:
len(set(aggregated_compulsory_courses))

824

In [ ]:
list(set(aggregated_compulsory_courses))[:10]

['',
 'Law and Governance of Artificial Intelligence',
 'Spatial Computing',
 'Exploratory Data Analysis and Visualization',
 'Introduction to Modern AI',
 'Database Management and SAS High Performance Computing with DBMS',
 'Data and Visual Analytics',
 'R-Programming in Banking and Finance-2',
 'Applied Regression Analysis',
 'HR analytics']

## Different approach to GDS labeling

In [ ]:
curr_india.head(2)

,url,compulsory course,course outcome or overview,topics covered
0,http://cds.iisc.ac.in/admissions/course-degrees/,Introduction to Scalable Systems,This course covers computer systems topics tha...,"Architecture: computer organization, single-co..."
1,http://cds.iisc.ac.in/admissions/course-degrees/,Numerical Linear Algebra,NaN,"Matrix and vector norms, floating points arith..."


In [ ]:
curr_usa.head(2)

,url,compulsory course,course outcome or overview,topics covered
0,https://www.seas.harvard.edu/applied-computati...,Systems Development for Computational Science,This is a project-based course emphasizing des...,NaN
1,https://www.seas.harvard.edu/applied-computati...,Introduction to Data Science,Data Science 1 is the first half of a one-year...,NaN


In [ ]:
def preprocess_text(text):

    # convert to lowercase
    text = text.lower()

    # remove digits
    text = text.translate({ord(k): None for k in string.digits})

    # remove punctuations
    text = text.translate({ord(k): None for k in string.punctuation})

    # remove URLs
    text = re.sub(r'http\S+', '', text)

    # remove unicode characters
    text = text.encode('ascii', 'ignore').decode('utf-8')

    # remove unnecessary whitespace
    text = text.replace('\t', '').replace('\n', '')
    text = re.sub(' +', ' ', text)
    text = text.strip()

    return text

In [ ]:
# combine compulsory courses from both countries
all_compulsory_courses = pd.concat([curr_india, curr_usa])

# clean data
all_compulsory_courses = all_compulsory_courses.fillna('\n')
all_compulsory_courses['course outcome or overview'] = all_compulsory_courses['course outcome or overview'].apply(preprocess_text)
all_compulsory_courses['compulsory course'] = all_compulsory_courses['compulsory course'].apply(preprocess_text)

# create a new feature with course details
all_compulsory_courses['course_details'] = all_compulsory_courses['course outcome or overview'] + all_compulsory_courses['topics covered']

# anonymize the courses by removing details about which programs they are a part of
all_compulsory_courses = all_compulsory_courses.drop(['url', 'course outcome or overview', 'topics covered'], axis=1)

# shuffle the courses
all_compulsory_courses = all_compulsory_courses.sample(frac=1)

all_compulsory_courses.shape

(1002, 2)

In [ ]:
all_compulsory_courses.head()

,compulsory course,course_details
562,data collection and cleaning,tools and techniques required for data collect...
348,legal aspects of business,\n
340,statistical learning,statistical learning is concerned with algorit...
361,business analytics lab,on successful completion of this course studen...
599,computer simulation,\n


In [ ]:
len(all_compulsory_courses['compulsory course'].unique())

796

In [ ]:
z = all_compulsory_courses.groupby(['compulsory course'], as_index=False).agg({'course_details': 'sum'})

In [ ]:
z

,compulsory course,course_details
0,,\n\n\n
1,accelerated regression analysis for business,this course is intended for students with rece...
2,accelerated statistics for public policy,\n
3,accounting and financial reporting,this course emphasizes the vocabulary methods ...
4,accounting for managers,\n\n\n
...,...,...
791,web analytics for business,explore web analytics text mining web mining a...
792,web intelligence and big data,\n
793,web technologies,advanced study of programming languages with a...
794,workforce planning and financial management,workforce planning and financial management pr...


In [ ]:
lol

,word,labels


In [ ]:
lol.loc[len(lol.index)] = ['blah', [1,2,3], 'h']

In [ ]:
lol

,word,labels,blah
0,NaN,NaN,1
1,NaN,NaN,2
2,NaN,NaN,3
3,blah,"[1, 2, 3]",h


In [ ]:
lol.loc[lol['word'] == 'blah', 'labels'] = 4

In [ ]:
lol

,word,labels,blah
0,NaN,NaN,1
1,NaN,NaN,2
2,NaN,NaN,3
3,blah,4,h


In [ ]:
f = ""
f += "45"
f += "56"
print(f)

4556


In [ ]:
d = [1,3,4,5]
d.remove(1)
d

[3, 4, 5]

In [ ]:
d[0]

3

In [ ]:
# create the csv file with all words

try:
    df = pd.read_csv("labelled_words.csv")
except FileNotFoundError:
    # load all necessary files
    data_path = "../../data_collection/data/processed"
    pgm_india = pd.read_csv(os.path.join(data_path, "india_pgms.csv"))
    pgm_usa = pd.read_csv(os.path.join(data_path, "usa_pgms.csv"))
    curr_india = pd.read_csv(os.path.join(data_path, "india_curr.csv"))
    curr_usa = pd.read_csv(os.path.join(data_path, "usa_curr.csv"))

    # aggregate names of compulosry courses
    compulsory_courses_india = curr_india['compulsory course'].fillna('')
    compulsory_courses_usa = curr_usa['compulsory course'].fillna('')
    aggregated_compulsory_courses = list(compulsory_courses_india) + list(compulsory_courses_usa)

    # process course names
    c = CourseNamesProcessor(aggregated_compulsory_courses)
    processed_course_names = c.get_courses()

    # extract unique words from course names
    processed_course_names_as_string = " ".join(processed_course_names)
    unique_words_in_course_names = list(set(processed_course_names_as_string.split()))

    # make dataframe
    df = pd.DataFrame(columns = ['word', 'labels'])
    df['word'] = unique_words_in_course_names
    df.to_csv("labelled_words.csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=596b9f0a-2431-4aa3-878c-95287ebfbe9a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>